In [ ]:
#Import necessay libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams["figure.figsize"] = (20,10)
import seaborn as sns

#Preprocessing
from sklearn import model_selection,metrics
from sklearn.preprocessing import OrdinalEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split,KFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OrdinalEncoder,LabelEncoder
#Model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,roc_auc_score
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import lightgbm as lgb
from sklearn.metrics import mean_squared_log_error,mean_squared_error
from catboost import CatBoostRegressor

In [ ]:
#import the data and shape
train = pd.read_csv("../input/tabular-playground-series-jan-2022/train.csv")
test = pd.read_csv("../input/tabular-playground-series-jan-2022/test.csv")
sample = pd.read_csv("../input/tabular-playground-series-jan-2022/sample_submission.csv")
print(train.shape,test.shape,sample.shape)
train.describe().transpose()

In [ ]:
train.sample(2)

In [ ]:
train.country.value_counts()

In [ ]:
train.store.value_counts()

In [ ]:
#insert the kfold columns
train['kfold'] = -1
#distributing the data
kfold = KFold(n_splits = 5,shuffle=True,random_state = 42)
for fold, (tr_i,va_i) in enumerate(kfold.split(X=train)):
    train.loc[va_i,'kfold'] = fold
    
print(train.kfold.value_counts())
train.to_csv("folds_5.csv",index=False)
print("successfully folds")

In [ ]:
train.columns,train.dtypes


In [ ]:
df = pd.read_csv("./folds_5.csv")

#features taken to train
usefulfeatures = [f for f in df.columns if f not in("row_id","date","kfold","num_sold")]
object_cols = [col for col in df.columns if col in ("product","store","country")]
df_test= test[useful_features]

In [ ]:
final_predictions = []
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    ytrain = xtrain.num_sold
    yvalid = xvalid.num_sold
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    ohe = preprocessing.OneHotEncoder(sparse=False, handle_unknown="ignore")
    xtrain_ohe = ohe.fit_transform(xtrain[object_cols])
    xvalid_ohe = ohe.transform(xvalid[object_cols])
    xtest_ohe = ohe.transform(xtest[object_cols])
    
    xtrain_ohe = pd.DataFrame(xtrain_ohe, columns=[f"ohe_{i}" for i in range(xtrain_ohe.shape[1])])
    xvalid_ohe = pd.DataFrame(xvalid_ohe, columns=[f"ohe_{i}" for i in range(xvalid_ohe.shape[1])])
    xtest_ohe = pd.DataFrame(xtest_ohe, columns=[f"ohe_{i}" for i in range(xtest_ohe.shape[1])])
    
    xtrain = pd.concat([xtrain, xtrain_ohe], axis=1)
    xvalid = pd.concat([xvalid, xvalid_ohe], axis=1)
    xtest = pd.concat([xtest, xtest_ohe], axis=1)
    
    # this part is missing in the video:
    xtrain = xtrain.drop(object_cols, axis=1)
    xvalid = xvalid.drop(object_cols, axis=1)
    xtest = xtest.drop(object_cols, axis=1)
    # missing part ends
     params_lgb = {
    "task": "train",
    "boosting_type": "gbdt",
    "objective": "binary",
    'subsample': 0.95312,
    'learning_rate': 0.001635,
    "max_depth": 3,
    "feature_fraction": 0.2256038826485174,
    "bagging_fraction": 0.7705303688019942,
    "min_child_samples": 290,
    "reg_alpha": 14.68267919457715,
    "reg_lambda": 66.156,
    "max_bin": 772,
    "min_data_per_group": 177,
    "bagging_freq": 1,
    "cat_smooth": 96,
    "cat_l2": 17,
    "verbosity": -1,
    'random_state':42,
    'n_estimators':5000,
    'colsample_bytree':0.1107
    }
    
    lgb_train = lgb.Dataset(xtrain, ytrain)
    lgb_val = lgb.Dataset(xvalid, yvalid)
    
    model = lgb.train(params=params_lgb,
                      train_set=lgb_train,
                      valid_sets=lgb_val,
                      early_stopping_rounds=300,
                      verbose_eval=False)
    
   
    preds_valid = model.predict(xvalid,num_iteration=model.best_iteration)
    test_preds = model.predict(xtest,num_iteration=model.best_iteration)
 
    final_predictions.append(test_preds)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

In [ ]:
#reconfigure of split data
final_predict = np.mean(np.column_stack(final_predictions),axis=1)
print(final_predict)
sample.num_sold = final_predict
sample.to_csv("submission_xgb.csv",index=False)
print("Final achieve to send xgboost output data")

## **Thankyou_Guys**
**Version 1 : Apply Catboost techniques**
**Version 2 : Apply Xgboost techniques**
## ***And explore More ? ComingSoon!!!!***